In [1]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [2]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('1_Merged_Data.csv')
booleandf = df.select_dtypes(include=[bool])
booleanDictionary = {True: 'TRUE', False: 'FALSE'}
for column in booleandf:
    df[column] = df[column].map(booleanDictionary)
df = df.drop(columns=["veh_lien_cnt"])
df = df.dropna()
df.dtypes

df

,hhld_id,drvr_cnt,veh_cnt,min_age,hoh_age,max_age,avg_age,min_mon_lic,hoh_mon_lic,max_mon_lic,...,naft_2012,naft_2013,naft_2014,naft_2015,naft_2016,aft_2012,aft_2013,aft_2014,aft_2015,aft_2016
0,4317,1,1,47.370047,47.370047,47.370047,47.370047,333.539214,290.637858,290.637858,...,0,0,0,0,0,0,0,0,0,1
2,3248,2,2,43.674743,48.459073,48.459073,46.066908,332.096922,389.508880,389.508880,...,0,0,2,0,0,0,0,1,0,0
3,2797,1,2,39.956548,39.956548,39.956548,39.956548,287.478578,287.478578,287.478578,...,0,1,0,0,0,0,0,0,0,0
4,1301,1,1,16.247208,16.247208,16.247208,16.247208,2.966494,2.966494,2.966494,...,0,0,1,0,0,0,1,0,0,0
5,264,1,1,31.147024,31.147024,31.147024,31.147024,181.764288,181.764288,181.764288,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2450,1206,1,1,35.802269,35.802269,35.802269,35.802269,237.627228,237.627228,237.627228,...,0,0,0,1,1,0,0,0,0,0
2452,431,1,2,39.564529,39.564529,39.564529,39.564529,282.774349,282.774349,282.774349,...,0,0,0,0,0,0,0,1,0,0
2453,3830,1,1,36.062151,36.062151,36.062151,36.062151,240.745810,240.745810,240.745810,...,0,0,1,0,0,0,0,0,0,0
2454,919,1,1,32.726766,32.726766,32.726766,32.726766,200.721196,200.721196,200.721196,...,1,0,0,0,0,0,0,0,0,0


In [4]:
train, test = train_test_split(df, test_size=0.2, random_state=4)
train, val = train_test_split(train, test_size=0.2)


print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

1459 train examples
365 validation examples
457 test examples


In [5]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('future_clm_ind')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds

In [6]:
feature_columns = []

num_features = ['min_age', 'max_age', 'avg_age', 'hoh_age', 'min_mon_lic','max_mon_lic',
      'avg_mon_lic', 'hoh_mon_lic', 'avg_majr_viol', 'prior_bi','time_w_carr','avg_minr_viol', 'credit_score']
cat_features = ['drvr_cnt', 'veh_cnt', 
       'cnt_yth', 'cnt_female', 'cnt_male', 'cnt_married', 'cnt_single',
       'cnt_auto', 'cnt_mtrcyc',
       'cnt_majr_viol', 'cnt_minr_viol', 'cnt_lic_susp', 
       'veh_lease_cnt', 'veh_own_cnt', 'monthly_pay_ind',
       'veh_w_coll_cnt', 'veh_w_comp_cnt', 'veh_w_ers_cnt', 'curnt_bi_low',
       'curnt_bi_upp', 'naft_2012', 'naft_2013', 'naft_2014',
       'naft_2015', 'naft_2016', 'aft_2012', 'aft_2013', 'aft_2014',
       'aft_2015', 'aft_2016']
ind_features = ['hoh_married', 'inforce_ind','fire_ind', 'homeowner_ind']

# numeric features
for header in num_features:
    feature_columns.append(feature_column.numeric_column(header))
    
# categorical features
for col_name in cat_features:
    categorical_column = feature_column.categorical_column_with_vocabulary_list\
    (col_name, df[col_name].unique())
    indicator_column = feature_column.indicator_column(categorical_column)
    feature_columns.append(indicator_column)
    
# indicator features

for col_name in ind_features:
    categorical_column = feature_column.categorical_column_with_vocabulary_list\
    (col_name, df[col_name].unique())
    indicator_column = feature_column.indicator_column(categorical_column)
    feature_columns.append(indicator_column)

feature_columns

[NumericColumn(key='min_age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='max_age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='avg_age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='hoh_age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='min_mon_lic', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='max_mon_lic', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='avg_mon_lic', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='hoh_mon_lic', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='avg_majr_viol', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='prior_bi', shape=(1,), default_value=None, dtype=

In [7]:
from tensorflow import keras

feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)



model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(140, activation='relu'),
  layers.Dense(100, activation='relu'),
  layers.Dense(60, activation='relu'),
  layers.Dropout(0.1),
  layers.Dense(1, activation='sigmoid')
])


model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(), \
              metrics=["accuracy"])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

Epoch 1/10

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

46/46 [==============================] - 2s 39ms/step - loss: 65.2120 - accuracy: 0.9164 - val_loss: 55.2956 - val_accuracy: 0.9260
Epoch 2/10
46/46 [==============================] - 0s 6ms/step - loss: 30.3020 - accuracy: 0.9184 - val_loss: 65.9656 - val_accuracy: 0.9260
Epoch 3/10
46/46 [==============================] - 0s 6ms/step - loss: 21.4365 - accuracy: 0.9102 - val_loss: 14.3625 - val_accuracy: 0.9178
Epoch 4/10
46/46 [==============================] - 0s 6ms/step - loss: 14.6728 -

In [8]:
from sklearn import metrics

y_test = test.future_clm_ind
y_pred = model.predict(test_ds)
print(metrics.accuracy_score(y_test, y_pred.round(), normalize=False)/457)
print(metrics.roc_auc_score(y_test,y_pred))

0.9409190371991247
0.8612350526503659
